In [1]:
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB
import random




# Parameters
with open("D:\\base\\jupyter\\gurobi\\example\\C1.1.txt" ) as f:
    txt = f.readlines()
num_customers = 100
num_facilities = 50
cartesian_prod = list(product(range(num_facilities), range(num_customers)))

costs = list(i.split(" ") for i in txt[2:])
costs = [list(map(eval,cost[:-1])) for cost in costs]
#固定成本
fixed_cost = [cost[1] for cost in costs ]
#运输成本
customs_to_facility = [cost[2:] for cost in costs]
shipping_cost = {(i,j): customs_to_facility[i][j] for i,j in cartesian_prod}    

#需求
demand = [random.randint(50,300) for j in range(num_customers)]
#服务能力
service = [random.randint(50,1000) for i in range(num_facilities)]

k=0

In [2]:
# MIP  model formulation

m = gp.Model('facility_location')

y = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
x = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='Assign')
z =  m.addVar( vtype=GRB.CONTINUOUS, name='z')

m.addConstr(gp.quicksum(demand[j] for j in range(num_customers)) <= \
                y.prod(service), name='Demand_Service')

m.setObjective(y.prod(fixed_cost)+z , GRB.MINIMIZE)
m.update()

Using license file D:\programs\special\GUROBI\gurobi.lic


In [3]:
#对偶子问题
#没有采用隐藏的约束：信息量大，结果会更好
ds = gp.Model("dual_subproblem")

α= ds.addVars(num_customers,lb=-GRB.INFINITY,ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name='a')
β = ds.addVars(cartesian_prod,lb=-GRB.INFINITY,ub=0,vtype=GRB.CONTINUOUS, name='b')

ds_constr = ds.addConstrs((α[j]+β[i,j]*demand[j] <= shipping_cost[i,j]*demand[j] for(i,j) in cartesian_prod),name='ds_constr')
ds.update()

In [4]:
def mycallback(model, where):
    global k
    
    
    if where == GRB.Callback.MIPSOL:
        y_relaxed = m.cbGetSolution(y)
        Z = m.cbGetSolution(z)
        
        ds.setObjective(gp.quicksum(α[j] for j in range(num_customers)) + \
               gp.quicksum(service[i]*y_relaxed[i]*β[i,j] for(i,j) in cartesian_prod)
               ,GRB.MAXIMIZE)
        ds.setParam(GRB.Param.OutputFlag,0)
        ds.optimize()
        z_ds = ds.ObjVal
        
        
        
        if (k<=5000) and (z_ds > Z):
            m.cbLazy(gp.quicksum(α[j].X for j in range(num_customers)) + \
                            gp.quicksum(service[i]*y[i]*β[i,j].X for(i,j) in cartesian_prod)  <= z )
            
            k += 1
            print("k:{}".format(k))

            
m.setParam(GRB.Param.LazyConstraints,1)          
m.setParam(GRB.Param.OutputFlag,0)
print("-----start-----")
m.optimize(mycallback)
print('Optimization complete')

print("objVal:{} \n time:{}".format(m.ObjVal,m.getAttr(GRB.Attr.Runtime)))

for i in range(num_facilities):
    if y[i].X==1:
        print("facilities_open:{}".format(i))
for j in range(num_customers):
    select = dict()
    value = list(x[i,j].X for i in range(num_facilities))
    for i in range(len(value)):
        if value[i] != 0:
            select[i] = value[i]
    print("customer_{}:{}".format(j,select))

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
-----start-----
k:1
k:2
k:3
k:4
k:5
k:6
k:7
k:8
k:9
k:10
k:11
k:12
k:13
k:14
k:15
k:16
k:17
k:18
k:19
k:20
k:21
k:22
k:23
k:24
k:25
k:26
k:27
k:28
k:29
k:30
k:31
k:32
k:33
k:34
k:35
k:36
k:37
k:38
k:39
k:40
k:41
k:42
k:43
k:44
k:45
k:46
k:47
k:48
k:49
k:50
k:51
k:52
k:53
k:54
k:55
k:56
k:57
k:58
k:59
k:60
k:61
k:62
k:63
k:64
k:65
k:66
k:67
k:68
k:69
k:70
k:71
k:72
k:73
k:74
k:75
k:76
k:77
k:78
k:79
k:80
k:81
k:82
k:83
k:84
k:85
k:86
k:87
k:88
k:89
k:90
k:91
k:92
k:93
k:94
k:95
k:96
k:97
k:98
k:99
k:100
k:101
k:102
k:103
k:104
k:105
k:106
k:107
k:108
k:109
k:110
k:111
k:112
k:113
k:114
k:115
k:116
k:117
k:118
k:119
k:120
k:121
k:122
k:123
k:124
k:125
k:126
k:127
k:128
k:129
k:130
k:131
k:132
k:133
k:134
k:135
k:136
k:137
k:138
k:139
k:140
k:141
k:142
k:143
k:144
k:145
k:146
k:147
k:148
k:149
k:150
k:151
k:152
k:153
k:154
k:155
k:156
k:157
k:158
k:159
k:160
k:161
k:162
k:163
k:164
k:165
k:166
k:167
k:1

k:1315
k:1316
k:1317
k:1318
k:1319
k:1320
k:1321
k:1322
k:1323
k:1324
k:1325
k:1326
k:1327
k:1328
k:1329
k:1330
k:1331
k:1332
k:1333
k:1334
k:1335
k:1336
k:1337
k:1338
k:1339
k:1340
k:1341
k:1342
k:1343
k:1344
k:1345
k:1346
k:1347
k:1348
k:1349
k:1350
k:1351
k:1352
k:1353
k:1354
k:1355
k:1356
k:1357
k:1358
k:1359
k:1360
k:1361
k:1362
k:1363
k:1364
k:1365
k:1366
k:1367
k:1368
k:1369
k:1370
k:1371
k:1372
k:1373
k:1374
k:1375
k:1376
k:1377
k:1378
k:1379
k:1380
k:1381
k:1382
k:1383
k:1384
k:1385
k:1386
k:1387
k:1388
k:1389
k:1390
k:1391
k:1392
k:1393
k:1394
k:1395
k:1396
k:1397
k:1398
k:1399
k:1400
k:1401
k:1402
k:1403
k:1404
k:1405
k:1406
k:1407
k:1408
k:1409
k:1410
k:1411
k:1412
k:1413
k:1414
k:1415
k:1416
k:1417
k:1418
k:1419
k:1420
k:1421
k:1422
k:1423
k:1424
k:1425
k:1426
k:1427
k:1428
k:1429
k:1430
k:1431
k:1432
k:1433
k:1434
k:1435
k:1436
k:1437
k:1438
k:1439
k:1440
k:1441
k:1442
k:1443
k:1444
k:1445
k:1446
k:1447
k:1448
k:1449
k:1450
k:1451
k:1452
k:1453
k:1454
k:1455
k:1456
k:1457

k:2486
k:2487
k:2488
k:2489
k:2490
k:2491
k:2492
k:2493
k:2494
k:2495
k:2496
k:2497
k:2498
k:2499
k:2500
k:2501
k:2502
k:2503
k:2504
k:2505
k:2506
k:2507
k:2508
k:2509
k:2510
k:2511
k:2512
k:2513
k:2514
k:2515
k:2516
k:2517
k:2518
k:2519
k:2520
k:2521
k:2522
k:2523
k:2524
k:2525
k:2526
k:2527
k:2528
k:2529
k:2530
k:2531
k:2532
k:2533
k:2534
k:2535
k:2536
k:2537
k:2538
k:2539
k:2540
k:2541
k:2542
k:2543
k:2544
k:2545
k:2546
k:2547
k:2548
k:2549
k:2550
k:2551
k:2552
k:2553
k:2554
k:2555
k:2556
k:2557
k:2558
k:2559
k:2560
k:2561
k:2562
k:2563
k:2564
k:2565
k:2566
k:2567
k:2568
k:2569
k:2570
k:2571
k:2572
k:2573
k:2574
k:2575
k:2576
k:2577
k:2578
k:2579
k:2580
k:2581
k:2582
k:2583
k:2584
k:2585
k:2586
k:2587
k:2588
k:2589
k:2590
k:2591
k:2592
k:2593
k:2594
k:2595
k:2596
k:2597
k:2598
k:2599
k:2600
k:2601
k:2602
k:2603
k:2604
k:2605
k:2606
k:2607
k:2608
k:2609
k:2610
k:2611
k:2612
k:2613
k:2614
k:2615
k:2616
k:2617
k:2618
k:2619
k:2620
k:2621
k:2622
k:2623
k:2624
k:2625
k:2626
k:2627
k:2628

k:3657
k:3658
k:3659
k:3660
k:3661
k:3662
k:3663
k:3664
k:3665
k:3666
k:3667
k:3668
k:3669
k:3670
k:3671
k:3672
k:3673
k:3674
k:3675
k:3676
k:3677
k:3678
k:3679
k:3680
k:3681
k:3682
k:3683
k:3684
k:3685
k:3686
k:3687
k:3688
k:3689
k:3690
k:3691
k:3692
k:3693
k:3694
k:3695
k:3696
k:3697
k:3698
k:3699
k:3700
k:3701
k:3702
k:3703
k:3704
k:3705
k:3706
k:3707
k:3708
k:3709
k:3710
k:3711
k:3712
k:3713
k:3714
k:3715
k:3716
k:3717
k:3718
k:3719
k:3720
k:3721
k:3722
k:3723
k:3724
k:3725
k:3726
k:3727
k:3728
k:3729
k:3730
k:3731
k:3732
k:3733
k:3734
k:3735
k:3736
k:3737
k:3738
k:3739
k:3740
k:3741
k:3742
k:3743
k:3744
k:3745
k:3746
k:3747
k:3748
k:3749
k:3750
k:3751
k:3752
k:3753
k:3754
k:3755
k:3756
k:3757
k:3758
k:3759
k:3760
k:3761
k:3762
k:3763
k:3764
k:3765
k:3766
k:3767
k:3768
k:3769
k:3770
k:3771
k:3772
k:3773
k:3774
k:3775
k:3776
k:3777
k:3778
k:3779
k:3780
k:3781
k:3782
k:3783
k:3784
k:3785
k:3786
k:3787
k:3788
k:3789
k:3790
k:3791
k:3792
k:3793
k:3794
k:3795
k:3796
k:3797
k:3798
k:3799

k:4830
k:4831
k:4832
k:4833
k:4834
k:4835
k:4836
k:4837
k:4838
k:4839
k:4840
k:4841
k:4842
k:4843
k:4844
k:4845
k:4846
k:4847
k:4848
k:4849
k:4850
k:4851
k:4852
k:4853
k:4854
k:4855
k:4856
k:4857
k:4858
k:4859
k:4860
k:4861
k:4862
k:4863
k:4864
k:4865
k:4866
k:4867
k:4868
k:4869
k:4870
k:4871
k:4872
k:4873
k:4874
k:4875
k:4876
k:4877
k:4878
k:4879
k:4880
k:4881
k:4882
k:4883
k:4884
k:4885
k:4886
k:4887
k:4888
k:4889
k:4890
k:4891
k:4892
k:4893
k:4894
k:4895
k:4896
k:4897
k:4898
k:4899
k:4900
k:4901
k:4902
k:4903
k:4904
k:4905
k:4906
k:4907
k:4908
k:4909
k:4910
k:4911
k:4912
k:4913
k:4914
k:4915
k:4916
k:4917
k:4918
k:4919
k:4920
k:4921
k:4922
k:4923
k:4924
k:4925
k:4926
k:4927
k:4928
k:4929
k:4930
k:4931
k:4932
k:4933
k:4934
k:4935
k:4936
k:4937
k:4938
k:4939
k:4940
k:4941
k:4942
k:4943
k:4944
k:4945
k:4946
k:4947
k:4948
k:4949
k:4950
k:4951
k:4952
k:4953
k:4954
k:4955
k:4956
k:4957
k:4958
k:4959
k:4960
k:4961
k:4962
k:4963
k:4964
k:4965
k:4966
k:4967
k:4968
k:4969
k:4970
k:4971
k:4972